In [ ]:
# Define a generator for all possible local views in cubic graphs,
# represented as a pair of lists. The first list comprises the colors 
# of the boundary: external neighbors of each u in N(v). The second 
# list comprises the edges inside N(v) = [1, 2, 3]
import itertools as its
from sage.combinat.permutation import Permutations_mset

# helper functions to manupulate colorings of the boundary
def color_Nus(Nu_sizes, tc):
    indices, _ = reduce(lambda (a, s), l: (a + [range(s, s+l)], s+l), Nu_sizes, ([], 0))
    return tuple(map(lambda Nu: map(lambda i: tc[i], Nu), indices))
def tc_hash(tc):
    s = int(len(tc)-1).bit_length()
    return sum(map(lambda (i, c): c<<(i*s), zip(its.count(0), tc)))
def permute_cNu(cNu, g_perm, c_perm):
    return map(lambda Nu: map(c_perm, Nu), [cNu[g_perm(i)] for i in range(len(cNu))])
is_minimal = lambda cNu: all(map(is_sorted, cNu))
is_sorted = lambda l: all(l[i] <= l[i+1] for i in xrange(len(l)-1))

def gen_LVs():
    # Iterate over possible graphs on N(v): empty, an edge, two edges, and a triangle 
    all_Nv_edges = [[], [[1,2]], [[1,2],[2,3]], [[1,2],[1,3],[2,3]]]
    all_Nus      = [[[4,5],[6,7],[8,9]], [[4],[5],[6,7]], [[4],[],[5]], [[],[],[]]]
    # Symmetry groups for the possible graphs (with zero-based indexing)
    all_sym_gps  = map(PermutationGroup, [[(0,1),(1,2)], [(0,1)], [(0,2)], [(0,1),(1,2)]])
    
    for Nv_edges, Nus, Nv_sym_gp in its.izip(all_Nv_edges, all_Nus, all_sym_gps):
        Nu_sizes = map(len, Nus)
      
        # There are 6 - 2*len(Nv_edges) vertices on the boundary to color
        for parts in Partitions(6 - 2*len(Nv_edges)):

            color_set = range(1, 1+len(parts))
            color_freqs = zip(color_set, parts)
            color_multiset = reduce(lambda a, t: a + [t[0]]*t[1], color_freqs, [])

            # Create the symmetry group for the colors
            def accum_gens(a, kg):
                colors = [f[0] for f in kg[1]]
                # Tuples representing generators for symmetric group on these colors
                new_gens = [tuple(colors[0:2]), tuple(colors)] if len(colors) > 1 else []
                return a + new_gens
            gens = reduce(accum_gens, its.groupby(color_freqs, lambda t: t[1]), [])
            color_gp = PermutationGroup(gens, domain=color_set)

            # Iterate through the isomorphic colorings
            found_tcs = set()
            for coloring in Permutations_mset(color_multiset):
                tc = tuple(coloring)
                cNu = color_Nus(Nu_sizes, tc)
                if not is_minimal(cNu):
                    continue # we only need 'minimal' colorings
                if tc_hash(tc) not in found_tcs:
                # When a new coloring is found, yield an LV and store
                # all isomorphic colorings to check against in future
                    yield [list(cNu), Nv_edges] 
                    for g_perm, c_perm in its.product(Nv_sym_gp, color_gp):
                        iso_cNu = map(sorted, permute_cNu(cNu, g_perm, c_perm))
                        found_tcs.add(tc_hash(flatten(iso_cNu)))
        

In [ ]:
# Name variables for the partition functions
q, r, s, t, varDelta = var('q, r, s, t, varDelta')
lam = var('lam', latex_name='\lam')
b = var ('b', latex_name='\beta')

# Partition functions and internal energy per particle for K33 and K4
ZK33(q, b) = q*(exp(-3*b)+q-1)^3+3*q*(q-1)*(exp(-2*b)+exp(-b)+q-2)^3 + \
              q*(q-1)*(q-2)*(3*exp(-b)+q-3)^3
UK33(q, b) = -ZK33(q, b).derivative(b)/ZK33(q, b)/6

ZK4(q, b) = q*exp(-6*b) + 4*q*(q-1)*exp(-3*b) + 3*q*(q-1)*exp(-2*b) + \
             6*q*(q-1)*(q-2)*exp(-b) + q*(q-1)*(q-2)*(q-3)
UK4(q, b) = -ZK4(q, b).derivative(b)/ZK4(q, b)/4

# Put all the local views in a list
LVs = list(gen_LVs())

# Give names to LVs that occur in the optimizing graphs
C1 = [[[1, 1], [1, 1], [1, 1]], []]
C2 = [[[1, 2], [1, 2], [1, 2]], []]
K4 = [[[], [], []], [[1, 2], [1, 3], [2, 3]]]

# Helper functions for lists
first = lambda l: l[0]
rest = lambda l: l[1:]

# constQ returns true for a list which is empty, a singleton, 
# or consists of repetitions of one value
def constQ(l):
    if len(l) <= 1: return true
    else: return l[0] == l[1] and constQ(rest(l))
    
# nonNegativeListQ returns true for nonempty lists of nonnegative numbers
def nonNegativeListQ(l):
    return (len(l) > 0 and 
            reduce(lambda x,y: x and y, map(lambda c: c >= 0,l),True))

# Find the largest color used in a local view
def findqC(C):
    cols = flatten(C[0])
    if cols == []: return 0
    else: return max(cols)

# Count the monochromatic edges in local view C with local coloring chi
# There are three terms, monochromatic edges incident to v, those from
# u in N(v) to an external neighbor, and those inside N(v)
def mChi(C, chi):
    cv = first(chi)
    cN = rest(chi)
    return (cN.count(cv) + 
            sum(map(lambda j: C[0][j].count(cN[j]), range(0,3))) + 
            map(lambda l: constQ([cN[j-1] for j in l]), C[1]).count(true))

# Count the monochromatic edges incident to v
def mvChi(C, chi):
    cv = first(chi)
    cN = rest(chi)
    return cN.count(cv)

# Sum the number the monochromatic edges incident to u in N(v)
# In fact this simply counts all monochromatic edges in the local view, 
# and double counts any inside N(v)
def mNChi(C, chi):
    cv = first(chi)
    cN = rest(chi)
    return (mChi(C,chi) + 
            map(lambda l: constQ([cN[j-1] for j in l]), C[1]).count(true))

# validChi returns True if chi uses an initial seqment of {qC+1,...,q}
def validChi(chi, qC):
    return (Set(chi).intersection(Set(range(qC+1,max(chi)+1))) == 
            Set(range(qC+1,max(chi)+1)))

# zChi returns the total weight of (a class of) local colourings
# divide by Z to obtain the probability
def zChi(C, q, b, chi):
    qC = findqC(C)
    if validChi(chi, qC):
        return binomial(q-qC, max(max(chi)-qC, 0))*exp(-b*mChi(C, chi))
    else:
        return 0

# Partition function for a local view
def Z(C, q, b):
    chis = Tuples(range(1, findqC(C)+5),4).list()
    return sum(zChi(C, q, b, chi) for chi in chis)

# Internal energy per particle from the perspective of v
def Uv(C, q, b):
    chis = Tuples(range(1, findqC(C)+5),4).list()
    return 1/2/Z(C, q, b)*sum(mvChi(C, chi)*zChi(C, q, b, chi) for chi in chis)

# Internal energy per particle from the perspective of N(v)
def UN(C, q, b):
    chis = Tuples(range(1, findqC(C)+5),4).list()
    return 1/6/Z(C, q, b)*sum(mNChi(C, chi)*zChi(C, q, b, chi) for chi in chis)

# We solve DualConstraint == 0 on C1 (or C2) to obtain Delta
# Note the change of notation to lam = e^(-b)
def DualConstraint(C, q, lam, varDelta):
    b = -log(lam)
    return Uv(C, q, b) + varDelta*(UN(C, q, b) - Uv(C, q, b)) - UK33(q, b)

Delta(q,lam) = solve(DualConstraint(C1, q, lam, varDelta) == 0, \
                     varDelta)[0].rhs()

# Slack in the dual constraint
def Slack(C, q, lam):
    b = -log(lam)
    return Uv(C, q, b) + Delta(q, lam)*(UN(C, q, b) - Uv(C, q, b)) - UK33(q, b)

# Scaling and reparametrisation of the slack to make observations easier
def ScaledSlack(C, r, t):
    q = r+3
    lam = 1/(1+t)
    b = log(1+t)
    ans = (ZK33(q, b)*Z(C, q, b)*4*(1+t)^17*(3+3*t+t^2+r*(1+t)^2)^2
           /(3+r)/t^2*Slack(C, q, lam)).simplify_log()
    if ans == 0: return ans # sage can't factor '0'
    else: return ans.factor() 

# Subtract Uv for C from Uv for K4    
def UvComparison(C, q, b):
    return UK4(q,b) - Uv(C,q,b)

# Scaling and reparametrisation of the difference to make observations easier
def ScaledUvComparison(C, s, t):
    q = s+max(3, findqC(C))
    b = log(1+t)
    ans = (ZK4(q, b)*Z(C, q, b)*2*(1+t)^14/t^2* \
           UvComparison(C, q, b)).simplify_log()
    if ans == 0: return ans # sage can't factor '0'
    else: return ans.factor() 

# Row of useful observations for a local view
def ObsRow(C):
    b = -log(lam)
    print('Computing ObsRow for local view %s' % C)
    return [C, simplify(Z(C, q, b)), simplify(2*Z(C, q, b)*Uv(C, q, b)), 
            simplify(6*Z(C, q, b)*UN(C, q, b)), ScaledSlack(C, r, t), 
            ScaledSlack(C,-1,t) if findqC(C) <= 2 else false,
            ScaledUvComparison(C, s, t), 
            ScaledUvComparison(C, -1, t) if findqC(C) <= 2 else false]

# Function to check the required observations, should all print 'True'
def RequiredObservations(C):
    ss0 = '  ScaledSlack is zero...                                    %s'
    ss1 = '  ScaledSlack has nonnegative coefficients...               %s'
    ss2 = '  ScaledSlack at q=2 has nonnegative coefficients...        %s'
    dv0 = '  ScaledUvComparison is zero...                             %s'
    dv1 = '  ScaledUvComparison has nonnegative coefficients...        %s'
    dv2 = '  ScaledUvComparison at q=2 has nonnegative coefficients... %s'
    
    print('Testing local view %s' % C)
    
    qC = findqC(C)
    ss = ScaledSlack(C, r, t)
    if C == C1 or C == C2:
        print(ss0 % ss.is_zero())
    else:
        print(ss1 % nonNegativeListQ(ss.polynomial(QQ).coefficients()))
        if qC <= 2:
            print(ss2 % nonNegativeListQ(ss(r=-1).polynomial(QQ).coefficients()))
    
    dv = ScaledUvComparison(C, s, t)
    if C == K4:
        print(dv0 % dv.is_zero())
    else:
        print(dv1 % nonNegativeListQ(dv.polynomial(QQ).coefficients()))
        if qC <= 2:
            print(dv2 % nonNegativeListQ(dv(s=-1).polynomial(QQ).coefficients()))

In [ ]:
# Check the required observations
map(RequiredObservations, LVs);
print('Done')

In [ ]:
# Generate the observation table
ObsTable = map(ObsRow, LVs)
save(ObsTable, os.path.join(os.getcwd(), 'ObsTable'))
save(LVs, os.path.join(os.getcwd(), 'LVs'))
print('Done')